In [1]:
import pyarrow.parquet as pq
from pyiceberg.catalog import load_catalog

In [ ]:
! curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o ../data/yellow_tripdata_2023-01.parquet

In [2]:
df = pq.read_table("../data/yellow_tripdata_2023-01.parquet")

In [3]:
print(df)

pyarrow.Table
VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
----
VendorID: [[2,2,2,1,2,...,2,2,1,1,1],[1,2,2,2,2,...,1,1,1,2,2],...,[2,2,2,2,2,...,2,2,2,2,2],[2,2,2,2,2,...,2,2,2,2,2]]
tpep_pickup_datetime: [[2023-01-01 00:32:10.000000,2023-01-01 00:55:08.000000,2023-01-01 00:25:04.000000,2023-01-01 00:03:48.000000,2023-01-01 00:10:29.000000,...,2023-01-02 21:16:11.000000,2023-01-02 21:56:02.000000,2023-01-02 21:04:31.000000,2023-01-02 21:13:09.000000,2023-01-02 21:45:30.000000],[2023-01-02 21:49:54.000000,2023-01-02 21:17:06.000000,2023-01-02 21:35:06.000000,2023-01-02 21:18:43.000000,2023-01-02 21:24:42.00

In [4]:
catalog = load_catalog(
    name="hive",
    **{
        "type": "hive",
        "uri": "thrift://hive-metastore:9083",
        "warehouse": "s3a://iceberg/lakehouse",
        "s3.endpoint": "http://minio:9000",
        "s3.access-key-id": "minio",
        "s3.secret-access-key": "minio123",
        "s3.path-style-access": "true",
        "s3.region": "us-east-1",
    }
)

In [6]:
catalog.create_namespace("bronze")

In [8]:
table = catalog.create_table(
    "bronze.taxi_dataset",
    schema=df.schema,
    location="s3://iceberg/lakehouse/bronze/iceberg_nyc_taxi"
)

OSError: When getting information for key 'lakehouse/bronze/iceberg_nyc_taxi/metadata/00000-1447851c-aecb-49fb-bbe5-9766a76b5326.metadata.json' in bucket 'iceberg': AWS Error ACCESS_DENIED during HeadObject operation: No response body.

In [ ]:
table.append(df)
len(table.scan().to_arrow())